In [1]:

import sys
import os
from IPython.display import Markdown
sys.path.append('../../')

from percolate.utils import make_uuid
from percolate.services.embedded import DuckDBService, KuzuDBService
from percolate.models.p8.embedding_types import EmbeddingRecord
from percolate.models import Agent
from percolate.services.embedded import DuckDBService, KuzuDBService, IcebergModelCatalog
from percolate.models import Agent

# s = DuckDBService(Agent)
# s.register()

In [1]:
import duckdb
import uuid
import json
from pyiceberg.exceptions import NoSuchNamespaceError, NamespaceAlreadyExistsError, TableAlreadyExistsError
from pyiceberg.catalog.sql import SqlCatalog
from percolate.utils import logger, make_uuid
from pydantic import BaseModel
from percolate.models import AbstractModel,Agent
import typing
import pyarrow as pa
from percolate.utils.env import P8_EMBEDDED_DB_HOME 
from percolate.utils.types.pydantic import arrow_type_to_iceberg_type
from percolate.services.embedded import IcebergModelCatalog

In [2]:
import duckdb
 
# Connect to DuckDB
con = duckdb.connect(f"{P8_EMBEDDED_DB_HOME}/percolate.db")


df=con.execute(f""" SELECT * FROM read_parquet('/Users/sirsh/.percolate/storage/p8.db/Agent/data/*.parquet') LIMIT 5 """).fetch_df()
df

In [6]:
con.execute(f""" SELECT * FROM read_parquet('/Users/sirsh/.percolate/storage/py_embeddings.db/p8_Agent_embeddings/data/*.parquet') LIMIT 5 """).fetch_df()

In [17]:
EXPLAIN
SELECT *
FROM embeddings
ORDER BY array_inner_product(embedding, embed('some text'))
LIMIT 5;

In [5]:
from percolate.services.embedded import DuckDBService
a=DuckDBService(Agent).add_embeddings(df.to_dict('records'))
a

In [5]:
b

In [31]:
joined_data = con.execute("""
    SELECT a.*, e.* 
    FROM iceberg_scan('p8.db.Agent' ) a
    JOIN iceberg_scan('py_embeddings.db.p8_Agent_embeddings') e 
    ON a.id = e.id
""").fetchall()

In [16]:
ic=IcebergModelCatalog(Agent).create_table_for_model()
#ic.cat.create_table('p8.test',schema=Agent.to_arrow_schema())

In [17]:
ic.scan().to_arrow().to_pandas()

In [12]:
import pyarrow as pa
data =[ Agent(
        id=make_uuid('SearchAgent2'),
        name="SearchAgent",
        category="Research",
        description="Agent that performs web searches and summarizes results",
        spec={"capabilities": ["search", "summarize"]},
        functions={"search_web": "Performs web search using search APIs"}
    )]



In [13]:

IcebergModelCatalog(Agent).upsert_data(data)
#c.upsert_data(data)

In [ ]:
# Step 2: Create and upsert records
print("\nCreating test agents...")
agents = [
    Agent(
        id=make_uuid("SearchAgent"),
        name="SearchAgent",
        category="Research",
        description="Agent that performs web searches and summarizes results",
        spec={"capabilities": ["search", "summarize"]},
        functions={"search_web": "Performs web search using search APIs"}
    ),
    Agent(
        id=make_uuid("WriterAgent"),
        name="WriterAgent",
        category="Content",
        description="Agent that writes and edits content based on prompts",
        spec={"capabilities": ["write", "edit", "proofread"]},
        functions={"generate_content": "Creates content based on prompts"}
    )
]

# Insert agents
print("\nInserting agents...")
service.update_records(agents)

In [ ]:
embedding_results = service.add_embeddings(service.select())
embedding_results